# Ingestão de Dados e Configuração Central
Parametrização central + execução dos pipelines de coleta (CDI, preços, Ibovespa, fundamentos) e extra BRAPI.

In [12]:
import subprocess
import sys
import json
import pandas as pd
import importlib
from pathlib import Path
import src.core.config
importlib.reload(src.core.config)
from src.core.config import get_config

cfg = get_config()
ROOT = cfg.paths.root
PROCESSED = cfg.paths.data_processed
print(f'ROOT: {ROOT}')
print(f'Processed: {PROCESSED}')

def run_module(module: str) -> tuple[int, str]:
    result = subprocess.run([sys.executable, '-m', module], capture_output=True, text=True)
    return result.returncode, (result.stdout or '') + (result.stderr or '')

def load_csv(path: Path) -> pd.DataFrame:
    return pd.read_csv(path) if path.exists() else pd.DataFrame()

ROOT: /home/subtilizer/Workspace/unb-cca-mqac
Processed: /home/subtilizer/Workspace/unb-cca-mqac/data/processed


## Parâmetros (única célula de configuração)
Salva overrides em `configs/analysis_overrides.json` para o pipeline usar como fallback default.

In [13]:
params = {
    'ticker_principal': 'PETR4.SA',
    'ticker_mercado': '^BVSP',
    'tickers_pares': ['PRIO3.SA', 'CSAN3.SA', 'RECV3.SA'],
    'data_inicio': '2016-01-01',
    'data_fim': '2025-11-30',
    'janela_beta_meses': 60,
    'peso_valor': 0.30,
    'peso_qualidade': 0.40,
    'peso_risco': 0.30,
}
cfg.update_analysis(**params)
cfg.save_analysis_overrides(params)
cfg.analysis.to_dict()

{'ticker_principal': 'PETR4.SA',
 'ticker_mercado': '^BVSP',
 'tickers_pares': ['PRIO3.SA', 'CSAN3.SA', 'RECV3.SA'],
 'data_inicio': '2016-01-01',
 'data_fim': '2025-11-30',
 'janela_beta_meses': 60,
 'taxa_livre_risco': None,
 'premio_risco_mercado': None,
 'premio_risco_pais': None,
 'roace_atual': None,
 'pl_atual': None,
 'ev_ebitda_atual': None,
 'dividend_yield': None,
 'margem_ebitda': None,
 'rp_ratio': None,
 'peso_valor': 0.3,
 'peso_qualidade': 0.4,
 'peso_risco': 0.3}

## CDI

In [5]:
code, out = run_module('src.data.fetch_cdi')
print(out.strip())
df = load_csv(PROCESSED / 'cdi.csv')
print(df.shape)
df.head()

INFO:__main__:CDI carregado via SGS série 4389
INFO:__main__:CDI carregado (série 4389): 2586 linhas, 2016-01-01 00:00:00 a 2025-11-28 00:00:00, gaps brutos: 98, gaps pós-ffill: 0
(2586, 3)


,date,cdi_annual,cdi_daily
0,2016-01-01,0.1414,0.000525
1,2016-01-04,0.1414,0.000525
2,2016-01-05,0.1414,0.000525
3,2016-01-06,0.1414,0.000525
4,2016-01-07,0.1414,0.000525


## Fundamentais PETR4

In [6]:
code, out = run_module('src.data.fetch_fundamentals')
print(out.strip())
df = load_csv(PROCESSED / 'fundamentals_petr4.csv')
print(df.shape)
df.head()

INFO:src.core.brapi_loader:Buscando PETR4 com módulos: ['defaultKeyStatisticsHistoryQuarterly', 'incomeStatementHistoryQuarterly', 'balanceSheetHistoryQuarterly', 'financialDataHistoryQuarterly']
INFO:src.core.brapi_loader:Dados obtidos para PETR4
INFO:__main__:Fundamentos carregados: 64 linhas, 2009-12-31 00:00:00 a 2025-09-30 00:00:00
(64, 11)


,quarter_end,pe_ratio,pb_ratio,roe,ev_ebitda,dividend_yield,revenue,ebitda,net_income,total_debt,equity
0,2009-12-31,NaN,NaN,NaN,NaN,NaN,182833794000,NaN,33344072000,NaN,1.668931e+11
1,2010-03-31,NaN,NaN,NaN,NaN,NaN,50412070000,NaN,7796847000,NaN,NaN
2,2010-06-30,NaN,NaN,NaN,NaN,NaN,3219309000,NaN,539951000,NaN,NaN
3,2010-09-30,NaN,NaN,NaN,NaN,NaN,1107664000,NaN,794891000,NaN,NaN
4,2010-12-31,NaN,NaN,NaN,NaN,NaN,158534626000,NaN,26769756000,NaN,3.102233e+11


## Ibovespa

In [7]:
code, out = run_module('src.data.fetch_ibovespa')
print(out.strip())
df = load_csv(PROCESSED / 'ibovespa.csv')
print(df.shape)
df.head()

INFO:src.core.brapi_loader:Buscando cotação de %5EBVSP via Brapi
INFO:src.core.data_loader:Buscando dados de ^BVSP de 2016-01-01 a 2025-11-30
INFO:src.core.data_loader:Obtidos 2463 registros para ^BVSP
INFO:__main__:Ibovespa obtido via Yahoo Finance
INFO:__main__:Cobertura Ibovespa: 2463 linhas, 2016-01-04 00:00:00 a 2025-11-28 00:00:00, 123 dias úteis faltantes
(2463, 8)


,date,open,high,low,close,adjusted_close,volume,ticker
0,2016-01-04,43349.0,43349.0,42125.0,42141.0,42141.0,2976300,^BVSP
1,2016-01-05,42139.0,42534.0,42137.0,42419.0,42419.0,2557200,^BVSP
2,2016-01-06,42410.0,42410.0,41590.0,41773.0,41773.0,3935900,^BVSP
3,2016-01-07,41772.0,41772.0,40695.0,40695.0,40695.0,4032300,^BVSP
4,2016-01-08,40695.0,41218.0,40463.0,40612.0,40612.0,3221600,^BVSP


## Preços PETR4

In [8]:
code, out = run_module('src.data.fetch_prices')
print(out.strip())
df = load_csv(PROCESSED / 'prices_petr4.csv')
print(df.shape)
df.head()

INFO:src.core.brapi_loader:Buscando cotação de PETR4 via Brapi
INFO:src.core.brapi_loader:Cotação obtida: PETR4 @ R$31.37
INFO:src.core.brapi_loader:Obtidos 2490 registros históricos para PETR4
INFO:__main__:Preços obtidos via Brapi
INFO:__main__:Cobertura: 2471 linhas, 2016-01-04 00:00:00 a 2025-11-28 00:00:00, 115 dias úteis faltantes
(2471, 8)


,date,open,high,low,close,adjusted_close,volume,ticker
0,2016-01-04,6.57,7.03,6.55,6.87,1.8882,45962100,PETR4
1,2016-01-05,6.92,7.00,6.62,6.68,1.8359,29446700,PETR4
2,2016-01-06,6.53,6.54,6.40,6.40,1.7590,67507200,PETR4
3,2016-01-07,6.19,6.43,6.08,6.26,1.7205,57387900,PETR4
4,2016-01-08,6.38,6.45,6.13,6.27,1.7233,52100300,PETR4


## Extra BRAPI (todos módulos)

In [9]:
code, out = run_module('src.data.fetch_brapi_modules')
print(out.strip())
df = load_csv(PROCESSED / 'brapi_stats.csv')
print(df.shape)
df.head()

INFO:src.core.brapi_loader:Buscando PETR4 com módulos: ['summaryProfile', 'financialData', 'balanceSheetHistory', 'balanceSheetHistoryQuarterly', 'incomeStatementHistory', 'incomeStatementHistoryQuarterly', 'cashflowHistory', 'cashflowHistoryQuarterly', 'defaultKeyStatistics', 'defaultKeyStatisticsHistory', 'defaultKeyStatisticsHistoryQuarterly', 'financialDataHistory', 'financialDataHistoryQuarterly', 'valueAddedHistory', 'valueAddedHistoryQuarterly']
INFO:src.core.brapi_loader:Dados obtidos para PETR4
INFO:__main__:BRAPI full payload salvo em /home/subtilizer/Workspace/unb-cca-mqac/data/external/brapi/brapi_full_petr4.json
(1, 21)


,type,symbol,enterpriseValue,forwardPE,profitMargins,sharesOutstanding,bookValue,priceToBook,mostRecentQuarter,earningsQuarterlyGrowth,...,trailingEps,enterpriseToRevenue,enterpriseToEbitda,52WeekChange,lastDividendValue,lastDividendDate,ytdReturn,totalAssets,updatedAt,dividendYield
0,yearly,PETR4,4.214616e+11,11.33571,0.07601,12888732761,32.97151,0.99177,NaN,NaN,...,2.88469,0.86161,1.96785,-0.12262,0.30845,2025-09-22,-0.09644,1212038000000,2025-11-14,15.89324
